# Data Mining Versuch Clustering von Pokemon

* Autoren: Manuel Eberhardinger, Johannes Theodoridis

## Abgabe:

- **Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen und den entsprechenden Ausgaben.**
- **Das Notebook ist als .ipynb und als .html abzugeben.**
- **Klausurelevante Fragen sind Dokument "Fragenkatalog Datamining" zu finden.**
- Antworten auf Fragen im Notebook, Diskussionen und Beschreibung der Ergebnisse sind optional (aber empfohlen) und werden nicht bewertet.

* [Übersicht Data Mining Praktikum](https://maucher.pages.mi.hdm-stuttgart.de/ai/page/dm/)


# Einführung

## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* Kennenlernen der Pokemon API
* Abfrage von Pokemon-Merkmalen über die API (Data Collection)
* Erstellen der Features für Pokemons (Feature Extraction + Data Preprocessing)
* Kennenlernen verschiedener Clustering-Algorithmen 
* Selektion der aussagekräftigsten Merkmale (Feature Selection)
* Clustering von ähnlichen Pokemons


## Vor dem Versuch zu klärende Fragen

### Kennenlernen der Pokemon API

In diesem Versuch wird die [Pokemon API](https://pokeapi.co/docs/v2) eingesetzt, um Features für verschiedene Pokemon über den [API Endpoint](https://pokeapi.co/docs/v2#pokemon) zu bekommen. Die API ist öffentlich verfügbar und man braucht somit keinen Account um Daten abfragen zu können. In diesem Versuch wollen wir keine fertige Library benutzen, die Daten für uns abfragen kann, sondern holen diese Daten mit der [requests](https://requests.readthedocs.io/en/latest/) Python-Bibliothek selber. 

1. Machen Sie sich mit der Pokemon API vertraut, speziell mit dem Pokemon endpoint. Diesen finden Sie über das Menü auf der Webseite: `Contents -> Pokémon -> Pokémon`.
2. Installieren Sie die Python-Bibliothek `requests` mit pip und führen Sie manuell ein paar beispielhafte Abfragen der Pokemon API durch. Benutzen Sie hierzu die `get` methode von requests mit einer endpoint URL. Eine Authentifizierung ist nicht notwendig. Versuchen Sie verschiedene IDs oder Namen aus und geben Sie den Statuscode sowie den Inhalt der response als Text oder JSON aus.
3. Welchen Python Datentyp liefert die Methoden `.json()` einer requests-Response zurück?

In [1]:

import requests as req
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import numpy as np
import pandas as pd
from requests import HTTPError, Timeout, TooManyRedirects
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [2]:
# Your Code
r = req.get("https://pokeapi.co/api/v2/ability/?limit=20&offset=0")
print(r.status_code)
print(r.text)

r = req.get("https://pokeapi.co/api/v2/pokemon-species/?limit=20&offset=0")
print(r.status_code)
print(r.text)

r = req.get("https://pokeapi.co/api/v2/pokemon-species/143/")
print(r.status_code)
print(r.text)

print(type(r.json()))

ConnectionError: HTTPSConnectionPool(host='pokeapi.co', port=443): Max retries exceeded with url: /api/v2/ability/?limit=20&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B74F8BAFD0>: Failed to establish a new connection: [WinError 10051] Ein Socketvorgang bezog sich auf ein nicht verfügbares Netzwerk'))

4. Schauen Sie sich nochmal das Kapitel zum unüberwachten Lernen der KI-Vorlesung an: [https://lectures.mi.hdm-stuttgart.de/mi7ai/06Clustering.html#introduction](https://lectures.mi.hdm-stuttgart.de/mi7ai/06Clustering.html#introduction) (**user**: *mi7ai*, **pw**: *ailecture*)
5. Beschreibt kurz die Clustering-Verfahren KMeans und DBSCAN. Was sind die jeweiligen Vor- und Nachteile der beiden Verfahren? Worin unterscheiden sie sich?

# Your Markdown (KMeans)
Pros:

K-means ist schnell und leicht zu verstehen.

Da der K-means Algorithmus garantiert ein locales Minimum des Reconstruction errors findet, wird es oft für aller Art Quantisierung benutzt.

K-means kann mit jeglichen Abstandsmetriken angewandt werden, wird jedoch im Beriech der Quantisierung, im Regelfall mit der Euklidischen Distanz verwendet.

Cons:

Die Anzahl der zufindenden Cluster muss vom Anwender vorgegeben werden, allerdings ist diese nicht immer leicht zubestimmen.(Was ist eine sinnvolle Anzahl an Cluster)

Das Resultat des Algorithmus hängt stark von den vordefinierten (zufälligen) Einstellungen ab.
Result depends on initial (random) cluster-setting?(warum Fragezeichen?)

Durch K-means werden alle Datenpunkte einem Cluster zugewiesen. Dadurch werden auche Ausreiser eine Cluster zugewiesen. Diese könnten ihr Cluster verformen.



# Your Markdown (DBSCAN)
Pros:

Man muss die Anzahl der Cluster nicht vor der Ausführung definieren.

Durch den DBSCAN werden Ausreiser keinem Cluster zugewiesen, somit können diese auch nicht die Cluster verformen.

Ist auch für größere Datensätze gut angepasst.

Cons:

Für diesn Algorithmus müssen zwei Variablen vordefiniert werden, was das Resultat des Algorithmus stark beeinflusst.

Das Resultat für eine gegebene Konfiguration ist einzigartig, da Rand-Punkte nicht nur zu einem Kern-punkt gehören müssen.

Unterschiede:

Konfigurationparameter: K-means --> Clusteranzahl; DBSCAN --> &#949; ,M mindest Anzahl der nahen Punkte um ein Kernpunkt zu sein

Beachten von Ausreisern: K-means --> ja, ordnet diese auch zu clustern hinzu ; DBSCAN --> nein, ignoriert diese und fügt sie keinem cluster hinzu


# Durchführung
## 1. Data Collection, Feature Extraction & Preprocessing

Als Erstes müssen wir Daten über die API holen, damit wir diese für das Erstellen der Features benutzen können. Wir arbeiten hier nur mit Daten über Pokemons und nicht mit den verschiedenen Fertigkeiten oder Attacken. Diese können für weitere eigene Experimente benutzt werden. Diesmal installieren wir keine spezielle Bibliothek, sondern holen uns die Daten selber über die [requests](https://requests.readthedocs.io/en/latest/) Bibliothek. Damit wollen wir euch zeigen, dass es auch ohne spezielle Frameworks sehr leicht ist, Daten in Python zu crawlen. Die meisten APIs arbeiten heutzutage mit JSON, es gibt aber auch noch Ausnahmen, die z.B. mit XML arbeiten. [JSON-Objekte](https://requests.readthedocs.io/en/latest/user/quickstart/#json-response-content) kann requests ohne Probleme selber parsen, da ein JSON-Objekt einfach in ein Python-Dictionary umgewandelt werden kannn. 

**Aufgaben 1:**
1. Implementiert eine Funktion `call_api(endpoint, resource_id)`, die euch für verschiedene endpoints Daten in einem Dictionary zurückgeben kann. Die endpoint URL muss dabei dynamisch um die id (oder den Namen) erweitert werden.
2. Um HTTP Fehler zu bemerken ruft auf dem Response-Objekt die Methode `raise_for_status()` auf damit wir mögliche Exceptions später korrekt abfangen können.
3. Zum Testen der Methode, holt euch die Daten für Pikachu über den Endpoint `https://pokeapi.co/api/v2/pokemon/{id or name}/`. Pikachu hat die id 25. Was passiert bei id 0? Analysiert die erste Ebene des Dictionaries mit der `.keys()` Methode.
4. Analysiert die Struktur bzw. die Datentypen der Keys `name, height, weight` sowie `stats` und `types` aus dem Dictionary mit der Beschreibung der API Dokumentation.
5. Gebt für Pikachu nun **manuell** den Name, Höhe und Gewicht, die Namen und Werte der Stats sowie die Typ(en) aus. Für Stats und Types ist eine geschachtelte Abfrage notwendig. Beachten Sie, dass manche Pokemons mehr als einen Typ haben können.

In [ ]:
# Your Code
def call_api(endpoint, resource_id):
    url = f"https://pokeapi.co/api/v2/{endpoint}/{resource_id}"
    r = req.get(url)
    
    rsponse_status = r.raise_for_status()
    return(r.json())




In [ ]:
# Test Cell

call_api("pokemon", 25)
# call_api("pokemon", 0) results in Error Code 404
print(call_api("pokemon", 25).keys())
print(type(call_api("pokemon", 25)['name'])) # name des Pokemons
print(type(call_api("pokemon", 25)['height'])) # Größe des Pokemons in Decimeter (0.1m)
print(type(call_api("pokemon", 25)['weight'])) # Gewicht des Pokemons in Hectogramm (100g)
print(type(call_api("pokemon", 25)['stats'])) # stats des Pokemons (hp, attack dmg,...)
print(type(call_api("pokemon", 25)['types'])) # the element type of the pokemon

print(call_api("pokemon", 25)['name'])
print(call_api("pokemon", 25)['height'])
print(call_api("pokemon", 25)['weight'])
print(call_api("pokemon", 25)['types'])
for value in call_api("pokemon", 25)['stats']:
    x = value['stat']['name']
    y = value['base_stat']
    print(f'{x} : {y}')
for value in call_api("pokemon", 25)['types']:
    print(value['type']['name'])


**Aufgabe 2:**

1. Implementiert nun eine Funktion `parse_poke_dict(pokemon)`, die das Dictionary, das von der `call_api`-Funktion zurückgegeben wurde, in ein Dictionary mit ausgewählten Features umwandelt (reduziert). Verwendet dafür die Keys und Features aus der vorherigen Aufgabe.
2. Das neue Dictionary soll nicht mehr verschachtelt sein. Einzelne Stats werden also auf der selben Hierarchieebene wie `name`, `height` und `weight` eingefügt. Verfügbare Types werden als Keys verwendet und auf den Value `1` gesetzt. Das ermöglicht uns später eine einfache Umwandlung der Types in ein binäres Encoding mit Pandas (ähnlich zu One-Hot Encoding, ein Pokemon kann aber mehr als einen Type haben).
3. Vergleichen Sie die neue Methode mit der manuellen Abfrage um sicherzustellen, dass alle Features korrekt übernommen wurden.

In [ ]:
# Your Code
def parse_poke_dict(pokemon):
    name = pokemon['name']
    height = pokemon['height']
    weight = pokemon['weight']
    hp = pokemon['stats'][0]['base_stat']
    attack = pokemon['stats'][1]['base_stat']
    defense = pokemon['stats'][2]['base_stat']
    special_attack = pokemon['stats'][3]['base_stat']
    special_defense = pokemon['stats'][4]['base_stat']
    speed = pokemon['stats'][5]['base_stat']
    reduced_dict = {
        "name": name,
        "height": height,
        "weight": weight,
        "hp": hp,
        "attack": attack,
        "defense": defense,
        "special-attack": special_attack,
        "speical-defense": special_defense,
        "speed": speed
        }

    for value in pokemon['types']:
        reduced_dict[value['type']['name']] = 1
    
    return reduced_dict
    
 

In [ ]:
# Test Cell
print(parse_poke_dict(call_api('pokemon', 25)))

**Aufgabe 3:**

1. Implementiert eine Funktion `get_img(pokemon)` welche ein noch nicht reduziertes Pokemon Dictionary akzeptiert und für dieses Pokemon das entsprechende Sprite (Bild) herunterlädt und zurück gibt. Hierzu ist ein weiterer `get`-Request notwendig.
2. Die Download URLs können über den Key 'sprites' abgfragt werden. Wir möchten die Ansicht 'front_default' herunterladen.
3. Der Inhalt der Response kann mit `.content` abgerufen werden. Um ein RGB-Bild zu erhalten muss dieser zunächst der Methode `BytesIO` und dann der Methode `Image.open` übergeben werden. Hierzu ist die `Pillow`-Bibliothek notwendig. Installiert diese also zunächst über pip und importiert dann das `Image`-Modul mit: `from PIL import Image`. Die `BytesIO`-Methode kann über die Python standard Bibliothek `io` importiert werden. Welchen Datentyp haben die Bilder die von der Methode zurückgegeben werden?
4. Für die Ausgabe der Bilder könnt ihr anstatt `print` die `display` Methode verwenden. Jupyter ruft diese implizit immer auf dem letzten Rückgabewert einer Zelle auf, daher werden Bilder angezeigt wenn sie der letzte Rückgabewert der Zelle waren.

In [ ]:
# Your Code
def get_img(pokemon):
    url = pokemon['sprites']['front_default']
    picture = req.get(url)
    return Image.open(BytesIO(picture.content))

In [ ]:
# Test Cell#
display(get_img(call_api('pokemon',25)))
# print(type(get_img(call_api('pokemon',25))))

**Aufgabe 4:**

Mit den bisher implementierten Methoden können wir nun unseren **Pokedex** erstellen. Dieser könnte zwar als Klasse implementiert werden, wir speichern die Daten und Bilder jedoch der Einfachheit halber separat in einem Pandas Dataframe und einem Numpy Array.

1. Implementiert eine Funktion `catch_them_all(n=151)`, welche für $n$ Pokemon die reduzierten Feature Dictionaries und Bilder sammelt. Wir verwenden $n=151$ als Default da das alle Pokemon aus der ersten Edition sind.
2. Verwendet die Methoden `call_api`, `parse_poke_dict` und `get_img` in einer Schleife um alle Pokemon abzufragen.
3. Die Dictionaries sollen zunächst in einer Python Liste gesammelt, und dann als Pandas Dataframe ausgegeben werden. Als Index soll dabei `name` verwendet werden. Was fällt bei der Betrachtung der kategorialen Einträge auf? Behebt das Problem in dem Ihr einen passenden Wert für fehlende Einträge setzt.
3. Die Bilder sammeln wir separat in einem Numpy Array. Da Numpy etwas picky sein kann wenn Objekte wie PNG Bilder abgespeichert werden sollen, erstellen wir die Listen nicht wie üblich mit `np.array([])` sondern mit `np.empty(shape=(n,), dtype=object)`. Um Objekte hinzuzufügen wird dann nicht wie üblich `np.append` verwendet sondern eine Zuweisung mit Index, also `a[i] = item`.
4. Der Grund warum wir hier ein Numpy array verwenden ist, dass wir später potentielle Cluster sehr einfach über eine Indize-Liste abfragen können, sowohl im Dataframe als auch im Array. Beispiel:

In [ ]:
import numpy as np
import pandas as pd

demo_data = pd.DataFrame(['Poke-1', 'Poke-2', 'Poke-3'])
demo_imgs = np.array(    ['IMG-1' , 'IMG-2' , 'IMG-3' ])

cluster_indices = [2,0]

display(demo_data.iloc[cluster_indices])
display(demo_imgs[cluster_indices])

In [ ]:
def catch_them_all(n):
    pokemon_list = []
    pokemon_pictures = np.empty(shape=(n,), dtype=object)
    for i in tqdm(range(1,n+1)):
        try :
            pokemon = call_api('pokemon', i)
            picture = get_img(pokemon)
        except (ConnectionError): 
            print(f"A Connection Error occurred at PokeID: {i}")
        except (HTTPError):
            print(f"A HTTPError occured at PokeID: {i}")
        except (Timeout): 
            print(f"A Timeout exception was thrown at PokeID: {i}")
        except (TooManyRedirects): 
            print(f"A TooManyRedirection Exception was thrown at PokeID: {i}")

        
        reduced = parse_poke_dict(pokemon)
        pokemon_list.append(reduced)
        pokemon_pictures[i-1] = picture

    print(len(pokemon_list))
    pokemon_df = pd.DataFrame(pokemon_list)
    pokemon_df = pokemon_df.fillna(0.0)
    pokemon_df = pokemon_df.set_index('name')
    return pokemon_df, pokemon_pictures

In [ ]:
pokemons,pictures = catch_them_all(151)

pokemons

4. Um eine Fortschrittsanzeige während des Downloads zu erhalten, installiert zunächst das Paket `tqdm`, importiert es dann wie folgt: `from tqdm.notebook import tqdm` und verwendet es in eurer Schleife, z.B. mit: `for i in tqdm(range(0, n)):`. Evtl. müssen noch die `ipywidgets` installiert werden.
5. Welche **Errors and Exceptions** können bei der Verwendung der [requests](https://requests.readthedocs.io/en/latest/user/quickstart/#errors-and-exceptions) Bibliothek auftreten? Listet diese in einer separaten Markdownzelle auf mit einer kurzen Erklärung der Fehler.

# Your Markdown

Connection Error, falls ein Netzwerkfehler auftritt.

HTTPError falls die Response.raise_for_status() methode einen unerfolgreichen Status code zurück sendet.

Timeout Exception im Falle eines Timeout bei einer Anfrage.

TooManyRedirects Exception, wenn bei einer Anfrage eine vorgegebene Anzahl an Weiterleitungen überstritten wird.


6. Wie kann sichergestellt werden, dass die Schleife bei einem Fehler nicht abbricht? Implementiert eine geeignete Lösung damit die Schleife auch mit requests-Exceptions weiterläuft. Informiert die Benutzer*innen der Methode bei welcher ID ein Fehler aufgetreten ist und gebt zusätzlich die Exception bzw. die Info, die der Error liefert. Tipp zum Testen: In Python können Exceptions manuell getriggert werden, z.B. mit `raise KeyError('Hallo, das ist ein KeyError')`, spezifische Exceptions einer Bibliothek müssen davor allerdings importiert werden. Deaktiviert alle manuell ausgelösten Test-Exceptions wieder soblad ihr euch sicher seid, dass diese robust abgefangen werden.
7. Testet euren *Pokedex* in dem ihr die Indizelisten `[0,1,2]` und `[24,150]` ausgeben lasst und stellt sicher, dass die Zuordnung zwischen Dataframe und Bild Array passt. Die `display`-Methode eignet sich hier wieder für die Ausgabe.

In [ ]:
# Your Code
display(pokemons.iloc[[0,1,2]], pictures[0], pictures[1], pictures[2])

display(pokemons.iloc[[24,150]],pictures[24],pictures[150])

In [ ]:
# Test Cell

In [ ]:
# Test Cell

Nachdem wir nun unser Dataset gesammelt haben sollten wir als Teil des Preprocessing noch sicherstellen, dass folgende Eigenschaften erfüllt sind: 

1. Keine Null- oder NaN-Werte im DataFrame.
2. Binäres Encoding der kategorischen Werte.
3. Skalieren der Werte mit einem MinMaxScaler.

Zwei davon haben wir schon als Teil der Feature Extraction erledigt. Um die Skalierung kümmern wir uns jetzt.

**Aufgabe 5:**

1. Instanziiert ein `from sklearn.preprocessing import MinMaxScaler` Objekt und wendet diesen auf den Dataframe an.
2. Die transformierten Daten sollen wieder in ein Pandas Dataframe umgewandelt werden. Die Columns sowie der Index sind dabei die Selben wie im unskalierten Dataframe.
3. Gebt die originalen und skalierten Daten aus, um die Dataframes zu vergleichen.
4. Importiert abschliesend die Bibliothek `seaborn` mit `import seaborn as sns` und analysiert die **skalierten numerischen Daten** mit einem [`sns.pairplot` (link)](https://seaborn.pydata.org/generated/seaborn.pairplot.html). Wählt dazu nur die numerischen Columns des Dataframes aus.
5. Beschreibt kurz Auffälligkeiten in den Verteilungen (falls vorhanden). 

In [ ]:
# Your Code
max_scaler = MinMaxScaler() 
data = pokemons
max_scaler.fit(data)
# print(max_scaler.data_max_)
scaled_data = max_scaler.transform(data)
# print(scaled_data)

scaled_df_pokemon = pd.DataFrame(scaled_data, index = data.index, columns = data.columns)
display(scaled_df_pokemon, pokemons)


In [ ]:
# Test Cell
sns.pairplot(scaled_df_pokemon.iloc[:,0:8])

In [ ]:
# Test Cell

An der 'height' und 'weight' Verteilung erkennt man das die meisten Pokemon klein und leicht sind.
'Attack','special-attack','special-defence' und 'speed'sind eher gleich verteilt.

## 2. Data Visualization & Dimensionsreduktion

Konntet ihr bereits Cluster in den Daten ausfindig machen? Nein? Keine Sorge, wir auch nicht. 

In diesem Abschnitt verwenden wir daher die Principal Component Analysis (PCA) Methode um unsere hochdimensionalen Daten in einem 2D-Raum abzubilden. Diese Methode wird in einer späteren Übung noch genauer erklärt und benutzt. Hier soll sie einfach als Blackbox verwendet werden. Man sollte nur wissen, dass damit ein hochdimensionaler Raum auf wenige Dimensionen reduziert werden kann ohne dabei viel Information zu verlieren. Auch das Clustering werden wir in diesem reduzierten Raum durchführen. Bevor wir uns aber um die PCA kümmern implementieren wir noch ein paar hilfreiche Visualisierungsfunktionen mit denen wir später die gefundenen Cluster besser analysieren können.

**Aufgabe 6:**

1. Implementiert eine Funktion `plot_images(img_list)` die als Input ein Numpy Array mit Pillow Image Objekten akzeptiert und diese in einem fixen Grid visualisiert. Verwendet dazu die Bibliothek `matplotlib` mit `import matplotlib.pyplot as plt`.
2. Erstellt zunächst **eine** neue Figure mit einer geeigneten quadratischen figsize. Pillow Images können mit der Funktion `imshow` geplottet werden. Um die Plots in einm Grid anzuordnen kann die Funktion `add_subplot(nrows, ncols, index)` verwendet werden (index muss bei 1 starten), bei 151 Pokemon ist ein 13x13 Grid eine gute Wahl. Für jeden Subplot können die Achsen mit `plt.axis('off')` deaktiviert werden.
3. Um eine Liste zu iterieren und dabei auch den aktuellen Index zu erhalten kann die Funktion `enumerate(['a','b','c'])` verwendet werden (Achtung: Index startet bei 0).
4. **Nachdem** alle Subplots erstellt wurden, muss **einmal** `plt.show()` aufgerufen werden.
5. Testet die Funktion mit dem Indize Subset `[0,1,2,24]` und einmal mit allen Pokemon.

In [ ]:
# Your Code
import matplotlib.pyplot as plt

def plot_images(img_list):
    fig = plt.figure(figsize =(13,13))
    img_list_enu = enumerate(img_list)
    for index, value in img_list_enu:
        fig.add_subplot(13,13,index+1)
        plt.axis('off')
        plt.imshow(value)
    plt.show()

In [ ]:
# Test Cell
plot_images(pictures[[0,1,2,24]])
plot_images(pictures)

**Aufgabe 7:**
    
1. Implementiert zwei Funktionen `plot_numerics(dataframe)` und `plot_categories(dataframe)` die als Input ein komplettes Dataframe akzeptieren aber jeweils nur die numerischen oder kategorischen Spalten visualisieren. Verwendet dazu wieder die Bibliothek `seaborn`.
2. Die Methoden sollen selbständig nur die numerischen oder kategorischen Spalten des Dataframes auswählen.
3. Für numerische Werte könnt ihr entweder ein [`sns.barplot` (link)](https://seaborn.pydata.org/generated/seaborn.barplot.html) oder [`sns.boxplot` (link)](https://seaborn.pydata.org/generated/seaborn.boxplot.html) verwenden. Damit wir später verschiedene Cluster miteinander vergleichen können ist eine konstante Y-Range sinnvoll, z.B. `plt.ylim(0,1.1)`.
4. Für kategorische Werte verwendet ihr ein `sns.barplot`. Per Default wendet seaborn hier den `estimator='mean'` auf die Daten an. Ist das für die kategorischen Columns eine geeignete Statistik? Mit welcher einfachen arithmetischen Funktion könnt ihr die Anzahl der binär codierten Kategorien zählen? Verwendet diese als estimator und wählt eine passende konstante Y-Range. Falls Fehlerbalken keine Sinn machen, können diese mit `errorbar=None` deaktiviert werden.
5. Die Labels der X-Achse können mit `plt.xticks(rotation=90)` rotiert werden.
6. Ruft in den Funktionen **noch nicht** `plt.show()` auf damit wir diese in einer Grid Darstellung verwenden können.
7. Testet eure Funktionen mit dem skalierten Dataframe.

In [ ]:
# Your Code
def plot_numeric(dataframe):
    columns = ['height', 'weight', 'hp', 'attack','defense', 'special-attack', 'speical-defense', 'speed']
    data = dataframe[columns]
    plt.ylim(-0.1,1.1)
    sns.boxplot(data)

def plot_categories(dataframe):
    data = dataframe.iloc[:,8:]
    plt.ylim(0,10.0)
    sns.barplot(data, estimator='sum')

In [ ]:
# Test Cell 
plot_numeric(scaled_df_pokemon)
plt.xticks(rotation=90)

In [ ]:
plot_categories(scaled_df_pokemon)
plt.xticks(rotation=90)

Die drei bisher implementierten Funktionen möchten wir nun dazu verwenden, um gefundene Cluster zu visualiseren. Mit der untenstehenden Funktion könnt Ihr euch zum Test Fake Cluster generieren. Später implementieren wir diese Datenstruktur mit den echten Clustern. Die Keys des Dictionaries sind die Cluster Namen, die Values jeweils eine Liste mit den Indizes der Pokemon, die zu diesem Cluster gehören.

In [ ]:
def get_fake_cluster(n_cluster=5, items_per_cluster=10, n_pokemon=151):

    cluster_dict = {}
    fake_cluster = np.random.choice(n_pokemon, (n_cluster, items_per_cluster), replace=False)
    for i, c in enumerate(fake_cluster):
        cluster_dict[i] = c
    return cluster_dict

test_cluster = get_fake_cluster()
test_cluster

**Aufgabe 8:**

1. Implementiert eine Funktion `compare_cluster(cluster_dict, dataframe, image_array)`, welche als Parameter ein Cluster Dictionary, die Pokemon Daten und Pokemon Bilder akzeptiert und für jedes Cluster die Daten und Bilder visualisiert. Wir vergleichen dabei zunächst die numerischen Werte der Cluster, dann die kategorischen und zum Schluss die Bilder. Geht bei der folgenden Implementierung Schritt für Schritt vor!
2. Erstellt zunächst eine Figure für die numerischen Werte mit der Größe `(20,40)`.
3. Nun iterieren wir über das `cluster_dict`. Da die Cluster Namen nicht zwingend bei 0 starten, ist es gut über den Inhalt des Dictionaries mit `for i, (c_name, c_indices) in enumerate(cluster_dict.items())` zu iterieren.
4. Erstellt für jedes Cluster ein Subplot in einem 10x5 Grid. Damit könnten bis zu 50 Cluster visualisiert werden. Dies ist erstmal ausreichend. Meistens haben wir viel weniger Cluster, die wir vergleichen möchten.
5. Gebt jedem Subplot einen Titel, der den Cluster Namen anzeigt.
6. Benutzt die Cluster Indizes nun um ein Subset des Dataframes zu selektieren und plottet das Cluster mit der `plot_numerics` Funktion.
7. Nach der Schleife ruft ihr `fig.tight_layout(h_pad=1, w_pad=1)` und `plt.show()` auf. Die Layout Funktion könnt ihr in Kombination mit der Größe der Figure verwenden, um die Darstellung schöner zu machen (sollte aber schon passen).
8. Wiederholt die Schritte 2-7, aber ruft dieses mal die Funktion `plot_categories` auf.
9. Iteriert ein letztes mal über das Cluster Dictionary, selektiert nun aber die Bilder und verwendet `plot_images`. Gebt mit einem einfachen `print`-Befehl davor an, welches Cluster gerade angezeigt wird.
10. Testet die Methode mit 1,5 und 10 Clustern. Wird alles korrekt dargestellt? Achtung, es gibt hier nichts inhaltliches zu analysieren. Das sind bisher nur Test Cluster ;) Zeigt bei der Abgabe, dass es für 10 Cluster gut funktioniert.

In [ ]:
# Your Code 
def compare_cluster(cluster_dict, dataframe, image_array):
        
    fig = plt.figure(figsize = (20,40))
    cluster_count = len(cluster_dict)
    for i, (c_name, c_indices) in enumerate(cluster_dict.items()):
        fig.add_subplot(10,cluster_count,i+1,title=c_name)
        plot_numeric(dataframe.loc[dataframe.index[c_indices]])
        plt.xticks(rotation=90)
        
    for i, (c_name, c_indices) in enumerate(cluster_dict.items()):
        fig.add_subplot(10,cluster_count,i+1+cluster_count,title=c_name)
        plot_categories(dataframe.loc[dataframe.index[c_indices]])
        plt.xticks(rotation=90)
    
        
    fig.tight_layout(h_pad=1, w_pad=1)
    plt.show()
    
    for i, (c_name, c_indices) in enumerate(cluster_dict.items()):
        fig.add_subplot(10,cluster_count,i+1+(2*cluster_count),title=c_name)
        plot_images(image_array[c_indices])
       


In [ ]:
test_cluster_2 = get_fake_cluster(10,10,151)
# compare_cluster(test_cluster_2,scaled_df_pokemon,pictures)

In [ ]:
# Test Cell
compare_cluster(test_cluster,scaled_df_pokemon,pictures)

Bisher können wir potentielle Cluster im hochdimensionalen Raum vergleichen, was uns erlaubt die Eigenschaften der gefundenen Cluster zu interpretieren. Für den Clustering-Algorithmus und die kompaktere Visualisierung wird empfohlen, eine Dimensionsreduktion mit der PCA-Methode durchzuführen, da dies eine gute Visualisierung der Daten in einem 2D-Raum ermöglicht.

**Aufgabe 9:**

1. Implementiert eine Funktion `apply_pca(dataframe)` welche ein skaliertes Dataframe akzeptiert und die Daten auf **zwei Dimensionen** reduziert.
2. Instanziiert ein [`from sklearn.decomposition import PCA` (link)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) Objekt. Wir möchten die Daten dabei auf zwei Komponenten reduzieren.
3. Wendet die PCA-Methode auf das Dataframe an. Die API ist dieselbe wie bei einem MinMaxScaler.
4. Die reduzierten Daten sollen wieder als Dataframe zurückgegeben werden. Der Index soll derselbe wie im ursprünglichem Dataframe sein.
5. Gebt den Dataframe vor und nach der Dimensionsreduktion aus. Mit der untenstehenden Funktion könnt ihr die auf zwei Dimensionen reduzierten Daten plotten. Der Index wird dabei als Hover Information angezeigt, sodass man erkennen kann welche Punkte wo liegen. Später können noch die Cluster Label und Cluster Center übergeben werden. Evtl. müsst ihr `plotly` installieren.

In [ ]:
from sklearn.decomposition import PCA

def apply_pca(dataframe):
    pca = PCA(n_components=2)
    reduced_data_2_components = pca.fit_transform(dataframe)

    reduced_df_pca = pd.DataFrame(data=reduced_data_2_components, index=dataframe.index)
    return reduced_df_pca


In [ ]:
display(scaled_df_pokemon)
x = apply_pca(scaled_df_pokemon)
x

In [ ]:
import plotly.express as px

def plot_pca_data(pca_dataframe, c_labels=None, c_center=None):
    '''
    This function plots a dataframe with two dimensions. If a list of cluster labels (and/or cluster centers)
    is provided, these are set as color highlights. The index of the dataframe is used as hover information for
    the labels. Note that the input dataframe is left unchanged as we work on a copy!
    '''
    df = pca_dataframe.copy()
    df['Cluster'] = c_labels
    df = df.sort_values(by='Cluster')
    df['Cluster'] = df['Cluster'].astype('str')
    fig = px.scatter(df, x=0, y=1, color='Cluster', hover_data={'Name': df.index})
    
    if c_center is not None:
        l, c = len(px.colors.qualitative.Plotly), len(c_center)
        df, colors = pd.DataFrame(c_center), (px.colors.qualitative.Plotly * ((c // l) + 1))[:c]
        fig.add_scatter(x=df[0], y=df[1], name="Cluster Center", mode='markers', marker={'symbol':
                       'cross-thin-open', 'size':10, 'color': colors, 'opacity':1})
    
    fig.show()

In [ ]:
# Your Code
plot_pca_data(x)

In [ ]:
# Test Cell

## 3. Clustering mit dem KMeans Algorithmus

Im letzten Teil des Versuchs wenden wir nun endlich ein Clustering Verfahren an. Wie ihr sehen werded, ist dies sehr einfach und auch der Grund, warum sich die ganze Vorarbeit auszahlt :)

**Aufgabe 10:**

1. Implementiert eine Funktion `kmeans(dataframe, n_cluster)`, die ein noch nicht reduzierten Dataframe, sowie die Anzahl der gewünschten Cluster akzeptiert. Als Rückgabewert erstellen wir ein Cluster Dictionary, das wir dann in der `compare_cluster`-Methode zur Analyse und Interpretation der Cluster verwenden können.
2. Instanziiert ein [`from sklearn.cluster import KMeans` (link)](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) Objekt mit `n_cluster`.
3. Wendet die Methode `apply_pca` auf den Dataframe an und fittet den KMeans-Algorithmus auf die transformierten Daten.
4. Ruft die Methode `plot_pca_data` auf und übergebt zusätzlich die Liste der gefundenen Cluster Labels. Optional kann zusätzlich noch die Liste der Clusterzentren übergeben werden.
5. Als letzten Schritt konvertieren wir noch die Liste der Cluster Labels in ein Cluster Dictionary. Implementiert diesen Schritt zunächst in einer separaten Code Zelle und integriert diesen dann in die `kmeans`-Methode, wenn ihr euch sicher seid, dass alles funktioniert.
6. Schaut euch zunächst die Liste an Cluster Labels an, die der KMeans-Algorithmus erzeugt. Diese möchten wir in ein Dictionary mit dem selben Format wie die Fake Cluster bringen. Die Keys sollen dabei die Cluster IDs sein, die Values jeweils eine Liste mit den Indizes der Elemente, die diesem Cluster zugeordnet wurden. Mit der Methode `np.unique` könnt ihr überprüfen welche eindeutigen Cluster IDs es gibt. Iteriert dann über diese Liste an IDs und verwendet `np.where`, um zu sehen welche Elemente in der Label Liste diesem Cluster zugeordnet wurden (ihr bekommt jeweils eine Liste mit den Indizes der Elemente zurück). Baut damit ein Cluster Dictionary und gebt dieses als Rückgabewert aus.
7. Zeigt beispielhaft das Ergebnis eines Clusterings und dass `compare_cluster` korrekt mit eurem Cluster Dictionary umgehen kann.

In [ ]:
# Your Code
from sklearn.cluster import KMeans

def kmeans(dataframe, n_cluster):
    kmeans = KMeans(n_clusters=n_cluster, random_state=42)
    transformed_df = apply_pca(dataframe)
    kmeans.fit(transformed_df)
    plot_pca_data(transformed_df, kmeans.labels_, kmeans.cluster_centers_)

    cluster_Ids = np.unique(kmeans.labels_)
    cluster_dict = {}
    for value in cluster_Ids:
        labels_of_the_cluster = np.where(kmeans.labels_ == value)
        cluster_dict[value] = labels_of_the_cluster
        
    return cluster_dict
    


In [ ]:
# Test Cell
cluster = kmeans(scaled_df_pokemon, 7)


In [ ]:
compare_cluster(cluster, scaled_df_pokemon, pictures)


**Aufgabe 11**:

1. Visualisiert euch die PCA für **numerische und kategorische Werte**. Wie viele Cluster erwartet ihr zu finden? 
2. Wendet den KMeans-Algorithmus mit dieser Anzahl auf die Daten an. Entspricht das Clustering euren Erwartungen?
3. Was ist für euch subjektiv gesehen eine optimale Anzahl an Clustern?
4. Analysiert und diskutieren die gefundenen Cluster mit den implementierten Methoden. Welche Merkmale oder Arten von Merkmalen haben eurer Meinung nach den größten Einfluss auf das Clustering?
5. Optional: Beschreibt die gefundenen Cluster mit einer kurzen Zusammenfassung, z.B. "Cluster 1: Große blaue Dreiecke".

In [ ]:
# Your Code

In [ ]:
# Your Markdown

**Aufgabe 12**:

1. Visualisiert euch die PCA nur für **numerische Werte**. Wie viele Cluster erwartet ihr zu finden? 
2. Wendet den KMeans Algorithmus mit dieser Anzahl auf die Daten an. Entspricht das Clustering euren Erwartungen?
3. Was ist für euch subjektiv gesehen eine optimale Anzahl an Clustern?
4. Analysiert und diskutieren die gefundenen Cluster mit den implementierten Methoden. Welche Merkmale oder Arten von Merkmalen haben eurer Meinung nach den größten Einfluss auf das Clustering?
5. Vergleicht die Ergebnisse mit denen aus Versuch 10.
5. Optional: Beschreibt die gefundenen Cluster mit einer kurzen Zusammenfassung, z.B. "Cluster 1: Kleine rote Kreise".

In [ ]:
# Your Code

In [ ]:
# Your Markdown

**Aufgabe 13:**

1. Berechnet für die gefundenen Cluster aus Aufgabe 11 und/oder 12 die [Elbow Methode](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html) und den [Silhouette Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).
2. Was genau berechnet die Elbow Methode und der Silhouette Score?
3. Passen die Scores zu eurer bisherigen Interpretation?

In [ ]:
# Your Code

In [ ]:
# Your Markdown

**Aufgabe 13: (Verständnisfragen)**

1. Was ist die maximale Anzahl an möglichen Clustern, wenn die numerischen Werte Teil des Dataframes sind?
2. Was ist die maximale Anzahl an möglichen Clustern, wenn nur die kategorischen Daten verwendet würden?
3. Wie beurteilt ihr die Verwendung von kategorischen Merkmalen im Clustering? Konnten dadurch Cluster gefunden werden, die ihr ohne die Hilfe des Algorithmus nicht so einfach gefunden hättet?
4. Wie beurteilen Sie die Verwendung von numerischen Merkmalen im Clustering? Konnten dadurch Cluster gefunden werden, die ihr ohne die Hilfe des Algorithmus nicht so einfach gefunden hättet?

Begründet eure Antworten entweder theoretisch (nur Text) und/oder optional auch empirisch (mit Code).

In [ ]:
# Your Code (optional)

In [ ]:
# Your Markdown